In [1]:
# p.468 Chapter15 畳み込みニューラルネットワーク
import numpy as np

def convld(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, int(len(x)/s),s):
        res.append(np.sum(x_padded[i:i + w_rot.shape[0]] * w_rot))
    return np.array(res)

# テスト
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]
print('Convld Implementation:', convld(x, w, p=2, s=1))

print('Numpy Results:', np.convolve(x, w, mode='same'))

Convld Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]
Numpy Results: [ 5 14 16 26 24 34 19 22]


In [3]:
import numpy as np
import scipy.signal

def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1, ::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2 * p[0]
    n2 = X_orig.shape[1] + 2 * p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X_orig.shape[0],
            p[1]:p[1]+X_orig.shape[1]] = X_orig
    
    res = []
    
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0]) / s[0]) + 1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1]) / s[1]) + 1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0], j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))

X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]
print('Conv2d Implementation: \n', conv2d(X, W, p=(1, 1), s=(1, 1)))

print('Scipy Results:\n', scipy.signal.convolve2d(X, W, mode='same'))

Conv2d Implementation: 
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]
Scipy Results:
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


In [5]:
import scipy.misc
img = scipy.misc.imread('./example-image.png', mode='RGB')
print('Image shape:', img.shape)

print('Number of channels:', img.shape[2])

print('Image data type:', img.dtype)

print(img[100:102, 100:102, :])

Image shape: (252, 221, 3)
Number of channels: 3
Image data type: uint8
[[[179 134 110]
  [182 136 112]]

 [[180 135 111]
  [182 137 113]]]


/Users/nagahamanil/.pyenv/versions/3.6.7/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


In [8]:
import os
import struct
import numpy as np

def load_mnist(path, kind='train'):
    """MNISTデータをpathからロード"""
    # 引数に指定したパスを結合(ラベルや画像のパスを作成)
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte' % kind)
    
    # ファイルを読み込む：
    # 引数にファイル、モードを指定(rbは読み込みのバイナリモード)
    with open(labels_path, 'rb') as lbpath:
        # バイナリを文字列に変換：unpack関数の引数にフォーマット、8バイト分の
        # バイナリデータを指定してマジックナンバー、アイテムの個数を読み込む
        magic, n = struct.unpack('>II', lbpath.read(8))
        # ファイルからラベルを読み込み配列を構築：fromfile関数の引数に
        # ファイル、配列のデータ形式を指定
        labels = np.fromfile(lbpath, dtype=np.uint8)
        
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
        # 画像ピクセル情報の配列のサイズを変更
        # (行数：ラベルのサイズ、列数：特徴量の個数)
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2
    return images, labels

In [10]:
# データの読み込み
X_data, y_data = load_mnist('./', kind='train')
print('Rows: %d, Columns: %d' % (X_data.shape[0], X_data.shape[1]))
X_test, y_test = load_mnist('./', kind='t10k')
print('Rows: %d, Columns: %d' % (X_test.shape[0], X_test.shape[1]))
X_train, y_train = X_data[:50000,:], y_data[:50000]
X_valid, y_valid = X_data[50000:,:], y_data[50000:]
print('Training: ', X_train.shape, y_train.shape)
print('Validation: ', X_valid.shape, y_valid.shape)
print('Test Set: ', X_test.shape, y_test.shape)

Rows: 60000, Columns: 784
Rows: 10000, Columns: 784
Training:  (50000, 784) (50000,)
Validation:  (10000, 784) (10000,)
Test Set:  (10000, 784) (10000,)


In [11]:
def batch_generator(X, y, batch_size=64, shuffle=False, random_seed=None):
    
    idx = np.arange(y.shape[0])
    
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X = X[idx]
        y = y[idx]
        
    for i in range(0, X.shape[0], batch_size):
        yield(X[i:i+batch_size, :], y[i:i+batch_size])

In [12]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)
X_train_centered = (X_train - mean_vals) / std_val
X_valid_centered = X_valid - mean_vals
X_test_centered = (X_test - mean_vals) / std_val

In [34]:
import tensorflow as tf
import numpy as np

def conv_layer(input_tensor, name, kernel_size, n_output_channels,
              padding_mode='SAME', strides=(1, 1, 1, 1)):
    with tf.variable_scope(name):
        # 入力チャネルの個数(n_input_channels)を取得：
        # input_tensorの形状：[バッチ×幅×高さ×チャネル数]
        input_shape = input_tensor.get_shape().as_list()
        n_input_channels = input_shape[-1]
        
        weights_shape = (list(kernel_size) + [n_input_channels, n_output_channels])
        
        weights = tf.get_variable(name='_weights', shape=weights_shape)
        print(weights)
        biases = tf.get_variable(
            name='_biases', initializer=tf.zeros(shape=[n_output_channels]))
        print(biases)
        
        conv = tf.nn.conv2d(input=input_tensor,
                           filter=weights,
                           strides=strides,
                           padding=padding_mode)
        print(conv)
        conv = tf.nn.bias_add(conv, biases, name='net_pre-activation')
        print(conv)
        conv = tf.nn.relu(conv, name='activation')
        print(conv)
        
        return conv

In [35]:
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    conv_layer(x, name='convest', kernel_size=(3, 3), n_output_channels=32)
    
del g, x

Tensor("convest/_weights/read:0", shape=(3, 3, 1, 32), dtype=float32)
Tensor("convest/_biases/read:0", shape=(32,), dtype=float32)
Tensor("convest/Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convest/net_pre-activation:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convest/activation:0", shape=(?, 28, 28, 32), dtype=float32)


In [36]:
def fc_layer(input_tensor, name, n_output_units, activation_fn=None):
    with tf.variable_scope(name):
        input_shape = input_tensor.get_shape().as_list()[1:]
        n_input_units = np.prod(input_shape)
        
        if len(input_shape) > 1:
            input_tensor = tf.reshape(input_tensor, shape=(-1, n_input_units))
            
        weights_shape = [n_input_units, n_output_units]
        weights = tf.get_variable(name='_weights', shape=weights_shape)
        print(weights)
        biases = tf.get_variable(name='_biases', initializer=tf.zeros(shape=[n_output_units]))
        print(biases)
        layer = tf.matmul(input_tensor, weights)
        print(layer)
        layer = tf.nn.bias_add(layer, biases, name='net_pre-activation')
        print(layer)
        if activation_fn is None:
            return layer
        
        layer = activation_fn(layer, name='activation')
        print(layer)
        return layer

In [37]:
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    fc_layer(x, name='fctest', n_output_units=32, activation_fn=tf.nn.relu)
    
del g, x

Tensor("fctest/_weights/read:0", shape=(784, 32), dtype=float32)
Tensor("fctest/_biases/read:0", shape=(32,), dtype=float32)
Tensor("fctest/MatMul:0", shape=(?, 32), dtype=float32)
Tensor("fctest/net_pre-activation:0", shape=(?, 32), dtype=float32)
Tensor("fctest/activation:0", shape=(?, 32), dtype=float32)


In [38]:
def build_cnn():
    # Xとyのプレースホルダを作成
    tf_x = tf.placeholder(tf.float32, shape=[None, 784], name='tf_x')
    tf_y = tf.placeholder(tf.int32, shape=[None], name='tf_y')
    
    # xを4次元テンソルに変換：
    # [バッチサイズ、幅、高さ、1]
    tf_x_image = tf.reshape(tf_x, shape=[-1, 28, 28, 1], name='tf_x_reshaped')
    
    # one-hot エンコーディング
    tf_y_onehot = tf.one_hot(indices=tf_y, depth=10, dtype=tf.float32,
                            name='tf_y_onehot')
    
    # 第1層：畳み込み層1
    print('\nBuilding 1st layer: ')
    h1 = conv_layer(tf_x_image, name='conv_1',
                   kernel_size=(5, 5),
                   padding_mode='VALID',
                   n_output_channels=32)
    
    # 最大値プーリング
    h1_pool = tf.nn.max_pool(h1,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    
    # 第2層：畳み込み層2
    print('\nBuilding 2nd layer: ')
    h2 = conv_layer(h1_pool, name='conv_2',
                   kernel_size=(5, 5),
                   padding_mode='VALID',
                   n_output_channels=64)
    
    # 最大値プーリング
    h2_pool = tf.nn.max_pool(h2,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    
    # 第3層：全結合層1
    print('\nBuilding 3rd layer:')
    h3 = fc_layer(h2_pool,
                 name='fc_3',
                 n_output_units=1024,
                 activation_fn=tf.nn.relu)
    
    # ドロップアウト
    keep_prob = tf.placeholder(tf.float32, name='fc_keep_prob')
    h3_drop = tf.nn.dropout(h3, keep_prob=keep_prob, name='dropout_layer')
    
    # 第4層：全結合層2(線形活性化)
    print('\nBuilding 4th layer:')
    h4 = fc_layer(h3_drop,
                 name='fc_4',
                 n_output_units=10,
                 activation_fn=None)
    
    # 予測
    predictions = {
        'probabilities' : tf.nn.softmax(h4, name='probabilities'),
        'labels' : tf.cast(tf.argmax(h4, axis=1), tf.int32, name='labels')
    }
    
    # TensorBoardで計算グラフを可視化
    
    # 損失関数と最適化
    cross_entropy_loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=h4, labels=tf_y_onehot),
        name='cross_entropy_loss')
    
    # オプティマイザ
    optimizer = tf.train.AdamOptimizer(learning_rate)
    optimizer = optimizer.minimize(cross_entropy_loss, name='train_op')
    
    # 予測正解率を特定
    correct_predictions = tf.equal(predictions['labels'], tf_y,
                                  name='correct_preds')
    
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32),
                             name='accuracy')

In [43]:
def save(saver, sess, epoch, path='./model/'):
    if not os.path.isdir(path):
        os.makedirs(path)
        
    print('Saving model in %s' % path)
    saver.save(sess, os.path.join(path, 'cnn-model.ckpt'), global_step=epoch)
    
def load(saver, sess, path, epoch):
    print('Loading model from %s' % path)
    saver.restore(sess, os.path.join(path, 'cnn-model.ckpt-%d' % epoch))
    
def train(sess, training_set, validation_set=None, initialize=True,
         epochs=20, shuffle=True, dropout=0.5, random_seed=None):
    X_data = np.array(training_set[0])
    y_data = np.array(training_set[1])
    training_loss = []
    
    # 変数を初期化
    if initialize:
        sess.run(tf.global_variables_initializer())
        
    # batch_generatorでシャッフルするため
    np.random.seed(random_seed)
    
    for epoch in range(1, epochs+1):
        batch_gen = batch_generator(X_data, y_data, shuffle=shuffle)
        
        avg_loss = 0.0
        for i, (batch_x, batch_y) in enumerate(batch_gen):
            feed = {'tf_x:0': batch_x,
                   'tf_y:0': batch_y,
                   'fc_keep_prob:0': dropout}
            loss, _ = sess.run(['cross_entropy_loss:0', 'train_op'],
                              feed_dict=feed)
            avg_loss += loss
            
        training_loss.append(avg_loss / (i+1))
        print('Epoch %02d Training Avg. Loss: %7.3f' %
             (epoch, avg_loss), end=' ')
        if validation_set is not None:
            feed = {'tf_x:0': validation_set[0],
                   'tf_y:0': validation_set[1],
                   'fc_keep_prob:0': 1.0}
            valid_acc = sess.run('accuracy:0', feed_dict=feed)
            print(' Validation Acc: %7.3f' % valid_acc)
        else:
            print()
            
def predict(sess, X_test, return_proba=False):
    feed = {'tf_x:0': X_test, 'fc_keep_prob:0': 1.0}
    if return_proba:
        return sess.run('probabilities:0', feed_dict=feed)
    else:
        return sess.run('labels:0', feed_dict=feed)

In [44]:
# ハイパーパラメータを定義
learning_rate = 1e-4 # 学習率
random_seed = 123 # 乱数シード

# 計算グラフを作成
g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    # モデルを構築
    build_cnn()
    # モデルを保存
    saver = tf.train.Saver()


Building 1st layer: 
Tensor("conv_1/_weights/read:0", shape=(5, 5, 1, 32), dtype=float32)
Tensor("conv_1/_biases/read:0", shape=(32,), dtype=float32)
Tensor("conv_1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer: 
Tensor("conv_2/_weights/read:0", shape=(5, 5, 32, 64), dtype=float32)
Tensor("conv_2/_biases/read:0", shape=(64,), dtype=float32)
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Building 3rd layer:
Tensor("fc_3/_weights/read:0", shape=(1024, 1024), dtype=float32)
Tensor("fc_3/_biases/read:0", shape=(1024,), dtype=float32)
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/net_pre-activation:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/act

In [45]:
# TensorFlosセッションの作成とCNNモデルのトレーニング
with tf.Session(graph=g) as sess:
    train(sess,
         training_set=(X_train_centered, y_train),
         validation_set=(X_valid_centered, y_valid),
         initialize=True,
         random_seed=123)
    save(saver, sess, epoch=20)

Epoch 01 Training Avg. Loss: 228.035  Validation Acc:   0.976
Epoch 02 Training Avg. Loss:  69.212  Validation Acc:   0.986
Epoch 03 Training Avg. Loss:  48.184  Validation Acc:   0.986
Epoch 04 Training Avg. Loss:  36.773  Validation Acc:   0.990
Epoch 05 Training Avg. Loss:  29.357  Validation Acc:   0.989
Epoch 06 Training Avg. Loss:  24.506  Validation Acc:   0.990
Epoch 07 Training Avg. Loss:  21.300  Validation Acc:   0.990
Epoch 08 Training Avg. Loss:  17.778  Validation Acc:   0.991
Epoch 09 Training Avg. Loss:  15.269  Validation Acc:   0.991
Epoch 10 Training Avg. Loss:  13.083  Validation Acc:   0.991
Epoch 11 Training Avg. Loss:  10.588  Validation Acc:   0.991
Epoch 12 Training Avg. Loss:   9.889  Validation Acc:   0.990
Epoch 13 Training Avg. Loss:   9.028  Validation Acc:   0.992
Epoch 14 Training Avg. Loss:   6.932  Validation Acc:   0.991
Epoch 15 Training Avg. Loss:   6.683  Validation Acc:   0.991
Epoch 16 Training Avg. Loss:   5.544  Validation Acc:   0.992
Epoch 17

In [46]:
# テストセットで予測正解率を計算
# 保存したモデルを復元

# 計算グラフを削除
del g

# 新しい計算グラフを作成し、モデルを構築
g2 = tf.Graph()
with g2.as_default():
    tf.set_random_seed(random_seed)
    build_cnn()
    saver = tf.train.Saver()
    
# 新しいセッションを作成し、モデルを復元
with tf.Session(graph=g2) as sess:
    load(saver, sess, epoch=20, path='./model/')
    # 予測正解率を計算
    preds = predict(sess, X_test_centered, return_proba=False)
    print('Test Accuracy: %.3f%%' % (100 * np.sum(preds == y_test) / len(y_test)))


Building 1st layer: 
Tensor("conv_1/_weights/read:0", shape=(5, 5, 1, 32), dtype=float32)
Tensor("conv_1/_biases/read:0", shape=(32,), dtype=float32)
Tensor("conv_1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv_1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer: 
Tensor("conv_2/_weights/read:0", shape=(5, 5, 32, 64), dtype=float32)
Tensor("conv_2/_biases/read:0", shape=(64,), dtype=float32)
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Building 3rd layer:
Tensor("fc_3/_weights/read:0", shape=(1024, 1024), dtype=float32)
Tensor("fc_3/_biases/read:0", shape=(1024,), dtype=float32)
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/net_pre-activation:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/act

In [47]:
# テストサンプルで予測を実行
np.set_printoptions(precision=2, suppress=True)

with tf.Session(graph=g2) as sess:
    load(saver, sess, epoch=20, path='./model/')
    print(predict(sess, X_test_centered[:10], return_proba=False))
    print(predict(sess, X_test_centered[:10], return_proba=True))

Loading model from ./model/
[7 2 1 0 4 1 4 9 5 9]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [48]:
# さらに20エポックのトレーニングを行う
# 初期化は再び行わに：initializer=False
# 新しいセッションを作成し、モデルを復元する
with tf.Session(graph=g2) as sess:
    load(saver, sess, epoch=20, path='./model/')
    
    train(sess,
         training_set=(X_train_centered, y_train),
         validation_set=(X_valid_centered, y_valid),
         initialize=False,
         random_seed=123)
    
    save(saver, sess, epoch=40, path='./model/')
    preds = predict(sess, X_test_centered, return_proba=False)
    print('Test Accuracy: %.3f%%' % (100 * np.sum(preds == y_test) / len(y_test)))

Loading model from ./model/
Epoch 01 Training Avg. Loss:   3.517  Validation Acc:   0.992
Epoch 02 Training Avg. Loss:   2.981  Validation Acc:   0.992
Epoch 03 Training Avg. Loss:   3.138  Validation Acc:   0.993
Epoch 04 Training Avg. Loss:   2.568  Validation Acc:   0.990
Epoch 05 Training Avg. Loss:   2.772  Validation Acc:   0.992
Epoch 06 Training Avg. Loss:   2.754  Validation Acc:   0.991
Epoch 07 Training Avg. Loss:   1.913  Validation Acc:   0.991
Epoch 08 Training Avg. Loss:   1.930  Validation Acc:   0.992
Epoch 09 Training Avg. Loss:   1.971  Validation Acc:   0.991
Epoch 10 Training Avg. Loss:   1.921  Validation Acc:   0.992
Epoch 11 Training Avg. Loss:   2.271  Validation Acc:   0.992
Epoch 12 Training Avg. Loss:   1.571  Validation Acc:   0.991
Epoch 13 Training Avg. Loss:   1.721  Validation Acc:   0.992
Epoch 14 Training Avg. Loss:   1.157  Validation Acc:   0.992
Epoch 15 Training Avg. Loss:   2.120  Validation Acc:   0.992
Epoch 16 Training Avg. Loss:   2.301  Vali

In [65]:
import tensorflow as tf
import numpy as np

class ConvNN(object):
    def __init__(self, batchsize=64, epochs=20, learning_rate=1e-4,
                dropout_rate=0.5, shuffle=True, random_seed=None):
        np.random.seed(random_seed)
        self.batchsize = epochs
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.shuffle = shuffle
        
        g = tf.Graph()
        with g.as_default():
            # 乱数シード(random-seed)を設定
            tf.set_random_seed(random_seed)
            # モデルを構築
            self.build()
            # 変数を初期化
            self.init_op = tf.grobal_variables_initializer()
            # saver
            self.saver = tf.train.Saver()
            
        # セッションを作成
        self.sess = tf.Session(graph=g)
        
    def build(self):
        # Xとyのプレースホルダを作成
        tf_x = tf.placeholder(tf.float32, shape=[None, 784], name='tf_x')
        tf_y = tf.placeholder(tf.int32, shape=[None], name='tf_y')
        is_train = tf.placeholder(tf.bool, shape=(), name='is_train')
        
        # xを4次元テンソルに変換：[バッチサイズ、幅、高さ、1]
        tf_x_image = tf.reshape(tf_x, shape=[-1, 28, 28, 1],
                               name='input_x_2dimages')
        
        # one-hotエンコーディング
        tf_y_onehot = tf.one_hot(indices=tf_y, depth=10, dtype=tf.float32,
                                name='input_y_onehot')
        
        # 第1層：畳み込み層1
        h1 = tf.layers.conv2d(tf_x_image, 
                              kernel_size=(5, 5), 
                              filters=32, 
                              activation=tf.nn.relu)
        
        # 最大値プーリング
        h1_pool = tf.layers.max_pooling2d(h1,
                                         pool_size=(2, 2),
                                         strides=(2, 2))
        
        # 第2層：畳み込み層2
        h2 = tf.layers.conv2d(h1_pool,
                             kernel_size=(5, 5),
                             filters=64,
                             activation=tf.nn.relu)
        
        # 最大値プーリング
        h2_pool = tf.layers.max_pooling2d(h2,
                                         pool_size=(2, 2),
                                         strides=(2, 2))
        
        # 第3層：全結合層1
        input_shape = h2_pool.get_shape().as_list()
        n_input_units = np.prod(input_shape[1:])
        h2_pool_flat = tf.reshape(h2_pool, shape=[-1, n_input_units])
        h3 = tf.layers.dense(h2_pool_flat, 1024, activation=tf.nn.relu)
        
        # ドロップアウト
        h3_drop = tf.layers.dropout(h3,
                                   rate=self.dropout_rate,
                                   training=is_train)
        
        # 第4層：全結合層2(線形活性化)
        h4 = tf.layers.dense(h3_drop,
                            10,
                            activation=None)
        
        # 予測
        predictions = {
            'probabilities': tf.nn.softmax(h4, name='probabilities'),
            'labels': tf.cast(tf.argmax(h4, axis=1), tf.int32, name='labels')
        }
        
        # 損失関数と最適化
        cross_entropy_loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=h4, labels=tf_y_onehot),
            name='cross_entropy_loss')
        
        # オプティマイザ
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        optimizer = optimizer.minimize(cross_entropy_loss, name='train_op')
        
        # 予測正解率を特定
        correct_predictions = tf.equal(predictions['labels'], tf_y,
                                      name='correct_preds')
        
        accuracy = tf.reduce_mean(tf.cast(correct_predictions,
                                         tf.float32),
                                 name='accuracy')
    
    def save(self, epoch, path='./tflayers-model/'):
        if not os.path.isdir(path):
            os.makedirs(path)
            
        print('Saving model in %s' % path)
        self.saver.save(self.sess,
                       os.path.join(path, 'model.ckpt'),
                       global_step=epoch)
        
    def load(self, epoch, path):
        print('Loading model from %s' % path)
        self.saver.restore(self.sess,
                          os.path.join(path, 'model.ckpt-%d' % epoch))
        
    def train(self, training_set, validation_set=None, initialize=True):
        # 変数を初期化
        if initialize:
            self.sess.run(self.init_op)
            
        self.train_cost_ = []
        X_data = np.array(training_set[0])
        y_data = np.array(training_set[1])
        
        for epoch in range(1, self.epochs + 1):
            batch_gen = batch_generator(X_data, y_data, shuffle=self.shuffle)
            avg_loss = 0.0
            for i, (batch_x, batch_y) in enumerate(batch_gen):
                feed = {'tf_x:0': batch_x,
                       'tf_y:0': batch_y,
                       'is_train:0': True} # ドロップアウト
                loss, _ = self.sess.run(['cross_entropy_loss:0', 'train_op'],
                                       feed_dict=feed)
                avg_loss += loss
                
            print('Epoch %02d: Training Avg. Loss: %7.3f' % (epoch, avg_loss), end=' ')
            
            if validation_set is not None:
                feed = {'tf_x:0': batch_x,
                       'tf_y:0': batch_y,
                       'is_train:0': False} # ドロップアウト
                valid_acc = self.sess.run('accuracy:0', feed_dict=feed)
                print('Validation Acc: %7.3f' % valid_acc)
            else:
                print()
                
    def predict(self, X_test, return_proba=False):
        feed = {'tf_x:0': X_test,
               'is_train:0': False} # ドロップアウト
        if return_proba:
            return self.sess.run('probabilities:0', feed_dict=feed)
        else:
            return self.sess.run('labels:0', feed_dict=feed)

In [66]:
cnn = ConvNN(random_seed=123)

## モデルのトレーニング
cnn.train(training_set=(X_train_centered, y_train),
         validation_set=(X_valid_centered, y_valid))
cnn.save(epoch=20)

AttributeError: module 'tensorflow' has no attribute 'layers'